In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Read the Excel files into the program. Excel only takes about a million rows so you will have to
#export them separately and then merge.
MyDF2017_1 = pd.read_excel('1.xlsx')

In [ ]:
MyDF2017_2 = pd.read_excel('2.xlsx')

In [3]:
#concatenate two dataframes
#MyDF2017 = pd.concat([MyDF2017_1, MyDF2017_2],ignore_index=True)
MyDF2017 = pd.read_csv('AllDataCSV.csv')
MyDF2017 = MyDF2017.drop_duplicates() #drop duplicate rows
x=MyDF2017.groupby(['npi'])['npi'].count()
print(len(x[x>1])) #double check there is no duplicate providers
#MyDF2017.to_excel('AllData.xlsx')

0


In [ ]:
#MyDF2017.to_csv('AllDataCSV.csv')

In [4]:
MyDF2017.columns
MyDF2017.set_index('npi')

,Unnamed: 0,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,0,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1003000142,1,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
1003000167,2,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
1003000175,3,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003000282,4,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,4.5148
1003000407,5,GIRARDI,DAVID,J,D.O.,M,I,100 HOSPITAL RD,NaN,BROOKVILLE,...,31.0,117.0,0.0,0.0,0.0,0.0,0.0,71.0,46.0,1.8494
1003000423,6,VELOTTA,JENNIFER,A,M.D.,F,I,11100 EUCLID AVE,NaN,CLEVELAND,...,0.0,NaN,NaN,0.0,NaN,0.0,0.0,48.0,17.0,0.7880
1003000480,7,ROTHCHILD,KEVIN,B,MD,M,I,12605 E 16TH AVE,NaN,AURORA,...,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.8413
1003000522,8,WEIGAND,FREDERICK,J,MD,M,I,1565 SAXON BLVD,SUITE 102,DELTONA,...,203.0,393.0,NaN,NaN,38.0,0.0,NaN,402.0,57.0,1.2057


In [5]:
x = MyDF2017.groupby(['bene_count'])['bene_count'].count()
sum((x.index * x))/sum(x) #determine average number of beneficiaries per provider (158)

158.34945851736759

In [7]:
x = MyDF2017['total_day_supply']/MyDF2017['total_claim_count']
x.median() #determine the median in a list of day per claim indexed by provider (29.7)

29.7125748502994

In [8]:
#group the database by specialty description and, for each specialty, determine the ratio of
#brand name drug claims to the total claims.
x = MyDF2017.groupby(['specialty_description'])[['specialty_description','total_claim_count',
                                                'brand_claim_count']].sum()
x = x[x['total_claim_count']>1000] #filter out categories w/ less than 1000 claims
x['brand_drug_fraction'] = x['brand_claim_count'] / x['total_claim_count'] #deteremine fraction
x['brand_drug_fraction'].std(skipna=True) #determine standard deviation (9.13%)

0.09132704930313962

In [9]:
#For each US state, determine the ratio of number of beneficiaries of opioids to that of 
#antibiotics.
x = MyDF2017.groupby(['nppes_provider_state'])[['nppes_provider_state','opioid_bene_count',
                                                'antibiotic_bene_count']].sum()
#Assuming only the US states are being considered; other US territories and countries are excluded.
x['opioid_antibiotic_ratio'] = x['opioid_bene_count']/x['antibiotic_bene_count']
exclude = ('XX','AA','AE','AP','AS','GU','MP','PR','VI','ZZ')
for item in x.index:
    if item in exclude:
        x.drop(item, inplace=True)
x['opioid_antibiotic_ratio'].max()-x['opioid_antibiotic_ratio'].min() #difference 0.446

0.44635372443082089

In [10]:
#Determine whether there is a correlation between the fraction of claim counts for 65+ beneficiaries
#and the fraction of claim counts.
x = MyDF2017[['total_claim_count','total_claim_count_ge65','ge65_suppress_flag','lis_claim_count',
         'lis_suppress_flag']].copy()
x['65_fraction'] = x['total_claim_count_ge65'] / x['total_claim_count']
x['lis_fraction'] = x['lis_claim_count'] / x['total_claim_count']
x.dropna(subset=['65_fraction','lis_fraction'],inplace=True)
x[['65_fraction','lis_fraction']].corr() #coefficient -0.6389

,65_fraction,lis_fraction
65_fraction,1.000000,-0.638932
lis_fraction,-0.638932,1.000000


In [11]:
#Determine the average day supply of opioids per claim. Then, for each state/specialty pair, 
#determine the average of this number. Finally, do some statistics on it.
x = MyDF2017[['npi','la_opioid_claim_count','la_opioid_day_supply', 'nppes_provider_state', 
              'specialty_description']].copy()
criterion = x['nppes_provider_state'].map(lambda a: a in exclude) #exclude data from non US states
x.drop(x[criterion].index, inplace=True)
x['avg_opioid_length'] = x['la_opioid_day_supply']/x['la_opioid_claim_count']
criterion = x['avg_opioid_length'].map(lambda a: a == np.nan)
x.drop(x[criterion].index, inplace=True) #drop NaN values

#create multi-index dataframe of provider count per state,specialty pair. Drop pairs w/
#less than 100 providers. Inner join it with similar multi-index of average length.
x_avglength = x.groupby(['nppes_provider_state','specialty_description'])['npi'].count()
x_avglength = x_avglength[x_avglength >= 100].to_frame()
x_avglength = x_avglength.join(x.groupby(['nppes_provider_state','specialty_description'])
                  ['avg_opioid_length'].mean(), how='inner')

#Create similar dataframe for average across all states. Use it in the previous dataframe.
ByState = x.groupby(['specialty_description'])['avg_opioid_length'].mean()
x_avglength['avg_opioid_ratio'] = np.nan
for state, descp in x_avglength.index:
    x_avglength.loc[(state, descp),'avg_opioid_ratio'] = x_avglength.loc[(state, descp),'avg_opioid_length']/ByState[descp]
x_avglength['avg_opioid_ratio'].max() #2.288 max ratio

2.2880137208395586

In [13]:
#Read in a new database for 2016 and store it in another dataframe.
MyDF2016_1 = pd.read_excel('2016 1.xlsx')
MyDF2016_2 = pd.read_excel('2016 2.xlsx')
MyDF2016 = pd.concat([MyDF2016_1, MyDF2016_2],ignore_index=True)
MyDF2016 = MyDF2016.drop_duplicates() #drop duplicate rows
x=MyDF2016.groupby(['npi'])['npi'].count()
print(len(x[x>1]))

0


In [59]:
#Always a smart idea to save the dataframe to CSV so that next time you do not have to wait
#for the slow read_excel method to load your dataframe.
MyDF2016.to_csv('AllDataCSV2016.csv')

In [14]:
#Determine the inflation in cost per day of drugs for each provider (from 2016 to 2017)

x = MyDF2017[['npi','total_day_supply','total_drug_cost']].copy().set_index('npi')
x.rename(columns = {'total_day_supply' : 'total_day_supply17', 
                    'total_drug_cost' : 'total_drug_cost17'}, inplace=True)
#Cost per day for each provider is assumed to be total aggregate drug cost divided by total aggregate days.
x['cost_per_day17'] = x['total_drug_cost17']/x['total_day_supply17']
x = x.join(MyDF2016[['npi','total_day_supply','total_drug_cost']].copy().set_index('npi'),how='inner')
x.rename(columns = {'total_day_supply' : 'total_day_supply16', 
                    'total_drug_cost' : 'total_drug_cost16'}, inplace=True)
x['cost_per_day16'] = x['total_drug_cost16']/x['total_day_supply16']
x['Inflation'] = (x['cost_per_day17'] - x['cost_per_day16'])/x['cost_per_day16']
x['Inflation'].describe()  
#mean is 0.2274 (22.74%). Sounds like a lot. Looks like there are a lot of unreasonably high values at the top 5% percentile pushing the average up.
#the median of 1.98% seems to be more representative. More investigation into these data may be needed.

count    1.054559e+06
mean     2.274707e-01
std      3.417497e+00
min     -1.000000e+00
25%     -2.101077e-01
50%     -1.986239e-02
75%      2.025134e-01
max      1.195490e+03
Name: Inflation, dtype: float64

In [15]:
#For each specialty, determine the fraction of specific providers that existed in 2016 and left in 2017.
x = MyDF2016.groupby(['specialty_description','npi'])['npi'].count().to_frame()
Specialty16 = MyDF2016.groupby('specialty_description')['specialty_description'].count()
Specialty16 = Specialty16[Specialty16>1000].to_frame().rename(columns={'specialty_description':'npi_16'})
#Specialty 16 contains counts of providers in 2016 in each category
x = x.join(MyDF2017.groupby(['specialty_description','npi'])['npi'].count().to_frame(),how ='inner',lsuffix='_16',rsuffix='_17')
Specialty17 = x.groupby(['specialty_description'])['npi_17'].count().to_frame()
#Specilatly17 uses inner join to determine the number of providers who left (which is associated with providers that left) not the net reduction.
Specialty16 = Specialty16.join(Specialty17, how='inner')
Specialty16['Fraction_Left']=(Specialty16['npi_16']-Specialty16['npi_17'])/Specialty16['npi_16']
Specialty16['Fraction_Left'].max()  ##37.69% max left

0.37686567164179102